In [1]:
import get_data_api as gda
import data_enrichment as de
import dim_tables as dim

import clean as cl
import transformation as tr
import create_bbdd as db

import pandas as pd

In [3]:
if __name__ == '__main__':
    db.main()

In [4]:
#get data from API
raw_users = gda.get_all_data('https://jsonplaceholder.typicode.com/users')

In [5]:
#get bookings data
raw_bookings = pd.read_csv('raw_bookings.csv', sep=';')

In [6]:
#create copy of raw_users
users_complete = raw_users.copy()

-Data enrichment-

Users data

In [ ]:
#enrich the user table to match the bookings table
users_complete = de.random_users(43562, users_complete)

In [7]:
#create copy of raw_users_complete
raw_users_complete = users_complete.copy()

In [8]:
#create companies id in users df
de.create_id(raw_users_complete, 'company_id')

In [9]:
#keep only 60% of companies
de.only_some_records(0.4, raw_users_complete, 'company_id', 'company')

In [11]:
raw_users_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          10 non-null     int64  
 1   name        10 non-null     object 
 2   username    10 non-null     object 
 3   email       10 non-null     object 
 4   address     10 non-null     object 
 5   phone       10 non-null     object 
 6   website     10 non-null     object 
 7   company     6 non-null      object 
 8   company_id  6 non-null      float64
dtypes: float64(1), int64(1), object(7)
memory usage: 848.0+ bytes


In [10]:
#create table raw_users and create ddbb


db.insert_table_sql(raw_users_complete, 'raw_users', "./bookings.db")

InterfaceError: Error binding parameter 4 - probably unsupported type.

Bookings data

In [ ]:
#add booking codes to all bookings
raw_bookings_complete = de.create_booking_id(raw_bookings)

In [ ]:
#add users_id to all bookings
de.create_id(raw_bookings_complete, 'user_id', len(raw_users_complete))

In [ ]:
raw_bookings_complete

In [ ]:
#save raw data
raw_bookings_complete.to_csv('raw_bookings_extended.csv', index=False)

raw_users_complete.to_csv('raw_users_extended.csv', index=False)

In [ ]:
#raw_bookings = pd.read_csv('raw_bookings_extended.csv')

In [ ]:
#raw_users_complete = pd.read_csv('raw_users_extended.csv')

-Create dimension tables- 

In [ ]:
#Create dimension agents
DIM_AGENTS = dim.create_dim_agent_table()

In [ ]:
#Create dimension countries
DIM_COUNTRIES = dim.create_dim_country_table()

In [ ]:
#Create dimension meals
DIM_MEALS = dim.create_dim_meal_table()

In [ ]:
#Create dimension hotels
DIM_HOTELS = dim.create_dim_hotel_table()

In [ ]:
#Create dimension users without transformation
DIM_USERS_WT = dim.create_dim_users_table(raw_users_complete)

In [ ]:
#Create dimension companies without transformation
DIM_COMPANIES_WT = dim.create_dim_companies_table(raw_users_complete)

-Transformation-

Users

In [ ]:
#delete unnecessary columns
DIM_USERS = tr.delete_columns(DIM_USERS_WT, ['username'])

In [ ]:
#lowercase some columns
DIM_USERS = tr.lowercase(DIM_USERS, ['email', 'website'])

In [ ]:
#capitalize column names and change them to standardized ones
DIM_USERS =  tr.capitalize_rename_columns(DIM_USERS, ['USER_ID', 'USER_NAME', 'USER_EMAIL','USER_ADDRESS', 'USER_PHONE', 'USER_WEBSITE', 'COMPANY_ID'])

In [ ]:
#cast company_id column to int
DIM_USERS = tr.cast_column_to_dtype(DIM_USERS,'COMPANY_ID', 'Int64')

In [ ]:
DIM_USERS

Companies

In [ ]:
#Delete rows containing NaN values
DIM_COMPANIES = tr.drop_nan_rows(DIM_COMPANIES_WT)

In [ ]:
#normalize company column
DIM_COMPANIES = tr.normalize_column(DIM_COMPANIES, ['company'])

In [ ]:
#delete unnecessary columns for analysis
DIM_COMPANIES = tr.delete_columns(DIM_COMPANIES, ['catchPhrase', 'bs'])

In [ ]:
#capitalize column names and change them to standardized ones
DIM_COMPANIES = tr.capitalize_rename_columns(DIM_COMPANIES, columns = ['COMPANY_ID', 'USER_ID','COMPANY_NAME'])

In [ ]:
#lowercase COMPANY_NAME column
DIM_COMPANIES = tr.lowercase(DIM_COMPANIES, ['COMPANY_NAME'])

In [ ]:
#cast company_id column to int
DIM_COMPANIES = tr.cast_column_to_dtype(DIM_COMPANIES,'COMPANY_ID', 'Int64')

In [ ]:
#sort column by company ids 
DIM_COMPANIES = DIM_COMPANIES.sort_values(by=['COMPANY_ID']).reset_index(drop=True)

In [ ]:
DIM_COMPANIES

Bookings

In [ ]:
#Replace the name of the hotels by their ids
BOOKINGS = tr.map_dimension_table(raw_bookings_complete,DIM_HOTELS,'hotel','HOTEL_ID','HOTEL_NAME')

In [ ]:
#cast agents column to int
BOOKINGS = tr.cast_column_to_dtype(BOOKINGS,'agent', 'Int64')

In [ ]:
#create a arrival date column
tr.create_arrival_date(BOOKINGS, 'arrival_date_day_of_month', 'arrival_date_month', 'arrival_date_year')

In [ ]:
#create a departure date column
tr.get_departure_date(BOOKINGS)

In [ ]:
#create a reservation date column
tr.reservation_date(BOOKINGS)

In [ ]:
drop_columns = ['is_canceled', 'lead_time',
       'arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month',
       'stays_in_weekend_nights', 'stays_in_week_nights','is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled']

#delete unnecessary columns for analysis
BOOKINGS = tr.delete_columns(BOOKINGS, drop_columns)

In [ ]:
rename_columns = ['BOOKING_CODE', 'HOTEL_ID', 'ADULTS', 'CHILDREN', 'MEAL_ID', 'COUNTRY_ID',
       'RESERVED_ROOM_TYPE', 'ASSIGNED_ROOM_TYPE', 'AGENT_ID',
       'STATUS', 'LAST_UPDATED_AT', 'USER_ID',
       'ARRIVAL_DATE', 'DEPARTURE_DATE', 'RESERVATION_DATE']

#capitalize column names and change them to standardized ones
BOOKINGS = tr.capitalize_rename_columns(BOOKINGS, rename_columns)

In [ ]:
#cast LAST_UPDATED_AT column to datetime
BOOKINGS['LAST_UPDATED_AT'] = pd.to_datetime(BOOKINGS['LAST_UPDATED_AT'], format='%d/%m/%Y')

In [ ]:
reorder_columns = [
 'BOOKING_CODE',
 'USER_ID',
 'COUNTRY_ID',
 'AGENT_ID',
 'HOTEL_ID',
 'ADULTS',
 'CHILDREN',
 'MEAL_ID',
 'RESERVED_ROOM_TYPE',
 'ASSIGNED_ROOM_TYPE',
 'STATUS',
 'LAST_UPDATED_AT',
 'ARRIVAL_DATE',
 'DEPARTURE_DATE',
 'RESERVATION_DATE'
]

#reorder columns
BOOKINGS = BOOKINGS[reorder_columns]

In [ ]:
BOOKINGS

-Clean data- 

In [ ]:
#put the prefix 'http://' in front of the websites that do not have it
DIM_USERS['USER_WEBSITE'] = DIM_USERS['USER_WEBSITE'].apply(cl.add_https)

In [ ]:
#remove 'mrs.' from USER_NAME
DIM_USERS['USER_NAME'] = DIM_USERS['USER_NAME'].str.replace('mrs. ','', regex = False)

In [ ]:
DIM_USERS

In [ ]:
def db():
    return sqlite3.connect('./bookings.db')

def create_table(table, name_sql_table):
    conn = db()
    table.to_sql(name_sql_table, conn, if_exists='replace', index = False)

In [ ]:
create_table(BOOKINGS, 'BOOKINGS')

In [ ]:
import sqlite3
conn = sqlite3.connect('./bookings.db')
BOOKINGS.to_sql('BOOKINGS', conn, if_exists='replace', index=False)
pd.read_sql('select * from BOOKINGS', conn)

In [ ]:
from sqlite3 import Error
cursor = conn.cursor()
try:
    sql_del = cursor.execute("DELETE FROM BOOKINGS")
    conn.commit()
except Error as e:
    print(f"Oops! Something went wrong. Error: {e}")
    # reverse the change in case of error
    conn.rollback()